In [190]:
def ConstructSidon(q, k, r): 
    ##Constructs a Sidon Space in Gq(rk, k) for r > 2 
    
    ##Define the field F_p^k and the polynomial ring over that field
    assert(r!=2)
    F.<z> = GF(q^k)
    R.<a> = F[]
    
    ##Construct another extension field of degree r over this ring using an irreducible polynomial of degree r over F_q^k
    irred_poly1 = R.irreducible_element(r)
    irred_poly2 = R.irreducible_element(r)
    F_r.<x> = F.extension(irred_poly1)[]
    F_ = F.extension(irred_poly1)
    ##find a root of this in F_q^rk
    roots = irred_poly2(x).roots()

    y = roots[0][0]
    
    ##returns a tuple containing y, the subfield F_q^k and the field F_q^rk
    ##The Sidon space is defined as u + u^p*y for u in F 
    return y, F, F_

In [191]:
def ConstructSidon2k(q, k): 
    ##Constructs a Sidon Space in Gq(2k, k), q cannot equal 2 
    assert(q!=2)
    
    ##Define the field F_p^k and the polynomial ring
    F.<z> = GF(q^k) 
    R.<a> = F[] 
    
    p = F.characteristic()
    ##Find and irreducible polynomial of degree 2 with the constant term not in W_q-1 
    irred_poly = R.irreducible_element(2)
    iterations = 0
   
    while irred_poly.list()[0]^((q^k - 1)/(q-1)) == 1 and iterations < 100: 
        irred_poly = R.irreducible_element(2)
        iterations += 1
    
    assert(iterations != 100)
    
  
    ##Construct the second extension field as a polynomial ring over the first modded out by this irreducible element
    X  = F.extension(irred_poly)
    F_2k.<x> = F.extension(irred_poly)[]
    roots = irred_poly(x).roots() 
    
    ##Return a root of the polynomial as well as info about the two fields 
    y = roots[0][0]
    
    return y, F, X, irred_poly.list()[1], irred_poly.list()[0]
    
    
    

In [182]:
def factor(product, y, q, F, F_r): 
    ##Factoring algorithm for Sidon spaces with r > 2 
    r = len(list(F_r.modulus())) - 1 
    p = F.characteristic()
    assert(r > 2)
    
    ##construct a basis from y 
    basis = [vector(y^n) for n in range(r)]
    mat = matrix(basis)
    
    ##calculate the change of basis matrix to go from the standard basis to our new basis, and compute 
    ##the representation of the product in that basis 
    cob_mat = mat.inverse() 
    product_representation = vector(product)*cob_mat
    
    ##Find the roots of the polynomial we derive from product_represntation, and then compute the original u and v
    assert(i == 0 for i in product_representation[3:])
    product_representation = product_representation[0:3]
    F_.<x> = F[]
    poly = F_(list(product_representation))
    roots = poly.roots()
    
    ##returns u and v up to multiplication from F_p, not the original things multiplied 
    if len(roots) == 1: 
        ans1 = (-1*1/roots[0][0]).nth_root(q - 1)
        ans2 = (-1*1/roots[0][0]).nth_root(q - 1)
        
    else: 
        ans1 = (-1*1/roots[0][0]).nth_root(F.characteristic() - 1)
        ans2 = (-1*1/roots[1][0]).nth_root(F.characteristic() - 1)
    
    if product/((ans1 + ans1^q*y)*(ans2 + ans2^q*y)) != 1: 
        ans1 = ans1*product/((ans1 + ans1^q*y)*(ans2 + ans2^q*y))
    return ans1, ans2

In [183]:
def factor2(product, y,q,F, F_r, b, c): 
    ##factoring algorithm for Sidon space with r = 2 
    p = F.characteristic()
    k = len(F.modulus().list()) - 1
    
    ##construct a basis for F_r that we will use to extract usefull info
    basis = [vector(y^n) for n in (0, 1)]
    mat = matrix(basis)
    cob_mat = mat.inverse() 
    product_representation = vector(product)*cob_mat
    
    
    ##Figure out the linear transformation x - cx^q 
    basis_transf = []
    identity = matrix.identity(k)
    
    for i in range(k): 
        transformed = vector(F(list(identity[i])) - c*F(list(identity[i]))^q)
        basis_transf += [transformed]
        
    transformation = matrix(basis_transf).inverse() 
    
    ##invert this transformation and use it to calculate uv     
    uv = vector(product_representation[0])*transformation
    uv = F(uv)
 
    ##Calculate the last two terms and create a quadratic polynomial
    F_.<s> = F[] 
    second_term = product_representation[1] + b*(uv)^q
    third_term = uv^q
    poly = F_([vector(uv), vector(second_term), vector(third_term)])
    
    ##get the roots of the polynomial and extract u and v
    roots = poly.roots()
    
    ##returns u and v up to multiplication from F_q, not the original things multiplied 
    if len(roots) == 1: 
        ans1 = (-1*1/roots[0][0]).nth_root(q - 1)
        ans2 = (-1*1/roots[0][0]).nth_root(q - 1)
        
    else: 
        ans1 = (-1*1/roots[0][0]).nth_root(q - 1)
        ans2 = (-1*1/roots[1][0]).nth_root(q - 1)
    
    if product/((ans1 + ans1^q*y)*(ans2 + ans2^q*y)) != 1: 
        ans1 = ans1*product/((ans1 + ans1^q*y)*(ans2 + ans2^q*y))
    return ans1, ans2
    

In [184]:
def publicKey(y,q, F, F_r): 
    ##Generates the public key (and associated private key) using info from the given sidon space 
    p = F.characteristic()
    k = len(F.modulus().list()) - 1
    rk = (len(F_r.modulus().list()) - 1)*k
    basefield = GF(q)
    iterations = 0
    iterations2 = 0
    
    ##Construct bases for the sidon space as well as F_r
    sidonbasis = Matrix(basefield, k, lambda i,j: basefield.random_element())
    while sidonbasis.is_invertible() == False and iterations < 100:
        sidonbasis = Matrix(basefield, k, lambda i,j: basefield.random_element())
        iterations += 1
    F_r_basis = Matrix(basefield, rk, lambda i,j: basefield.random_element())
    while F_r_basis.is_invertible() == False and iterations2 < 100: 
         F_r_basis = Matrix(basefield, rk, lambda i,j: basefield.random_element())
         iterations2 += 1
    
    assert(iterations<100 and iterations2<100)
    
    ##Get the multiplication table from the basis of the sidon space
    v = [F(list(sidonbasis[i])) for i in range(k)]
    origbasis = sidonbasis
    sidonbasis = [j + j^q*y for j in v]
    mult_table = vector(sidonbasis).column()*vector(sidonbasis).row()
    cob_matrix = F_r_basis.inverse()
    print(cob_matrix)
    vec_list = [[0 for i in range(k)] for j in range(k)]
    
    ##Generate the public key M(V,B)
    for i in range(k): 
        for j in range(k):
            element = mult_table[i][j]
            long_representation = convertToLong(element)
            vec_list[i][j] = list(long_representation*cob_matrix) 
    matrixlist = [0 for i in range(rk)]
    
    ##Return the public key 
    for i in range(rk): 
        matrixlist[i] = Matrix(basefield, k, lambda l,j: vec_list[l][j][i])
    return matrixlist, sidonbasis, mult_table, F_r_basis, origbasis
def convertToLong(element):
    ##Converts an element from a vector over F to a vector over F_q
    long_representation = []
    print(list(element))
    for l in range(len(list(element))): 
        long_representation += list(vector(list(element)[l]))
    long_representation = vector(long_representation)
    long_representation
    return long_representation

def convertFromLong(element, F, F_r):
    ##Converts an element from F_r represented as a vector over F_q to a vector over F
    
    k = len(F.modulus().list()) - 1
    r = len(F_r.modulus().list()) - 1
    final_list = []
    
    for i in range(r): 
        final_list += [F(list(element)[i*k:(i+1)*k])]
    return(F_r(final_list))
    
    

In [185]:
def encrypt(a,b, matrixlist): 
    ##Uses the public key to send the encrypted message
    anslist = []
    print(a,b,matrixlist)
    for i in matrixlist: 
        anslist += a.row()*i*b.column()
    return anslist 


def decrypt(anslist, y, q, F, F_r, F_r_basis, sidonbasis, origbasis, b = None, c = None): 
    ##Decrpyts the message anslist
    ##Calculate the original product in the sidon space
    product = (anslist[0]*convertFromLong(F_r_basis[0],F,F_r))
    
    for i in range(len(anslist) - 1): 
        product += anslist[i+1]*convertFromLong(F_r_basis[i+1], F, F_r)
    r = len(F_r.modulus().list()) - 1 
    
    ##Factor the product
    if r == 2: 
        u,v = factor2(product[0], y,q, F, F_r, b, c)
    else: 
        u,v = factor(product[0], y,q, F, F_r)
    print(product)
    print((u + u^q*y)*(v + v^q*y))
    
    ##Represent the product over the sidon space basis
    cob_matrix = origbasis.inverse() 
  
    u = vector(F(str(u)))*origbasis.inverse()
    v = vector(F(str(v)))*origbasis.inverse() 
    
   
    return u.column()*v.row()


        

In [186]:
##demonstration
q = 5
k = 3
r = 4
y, F, F_r, d, c = ConstructSidon2k(q, k)
matrixlist, sidonbasis, mult_table, F_r_basis , origbasis = publicKey(y,q,F,F_r)
a = vector(GF(q), [2,2,0])
b = vector(GF(q), [1,1,3])


[3 1 1 1 1 1]
[4 0 2 3 4 1]
[1 4 1 2 0 1]
[1 4 0 0 0 1]
[4 1 1 2 4 4]
[2 4 4 1 3 2]
[3*z^2 + 2*z, z^2 + 4*z + 3]
[z^2 + 4*z + 1, 4*z + 1]
[3*z + 3, 2*z + 1]
[z^2 + 4*z + 1, 4*z + 1]
[3*z^2 + 1, 2*z^2 + 3*z]
[4*z^2, 2*z^2 + 3*z + 1]
[3*z + 3, 2*z + 1]
[4*z^2, 2*z^2 + 3*z + 1]
[4*z^2 + 4*z + 2, z^2 + z + 2]


In [187]:
anslist = encrypt(a,b, matrixlist)

(2, 2, 0) (1, 1, 3) [[2 2 0]
[2 2 1]
[0 1 4], [2 3 4]
[3 4 1]
[4 1 1], [0 4 1]
[4 0 0]
[1 0 4], [1 3 1]
[3 0 1]
[1 1 0], [2 3 3]
[3 4 3]
[3 3 0], [1 3 0]
[3 0 1]
[0 1 3]]


In [189]:
decrypt(anslist, y,q, F, F_r, F_r_basis, sidonbasis, origbasis, d,c)

((3*z^2 + 2)*a + 3*z + 4)
(3*z^2 + 2)*a + 3*z + 4


[2 2 0]
[2 2 0]
[1 1 0]

In [47]:
anslist

[(2), (1), (1), (1), (1), (2), (2), (1), (1), (1), (1), (1)]

In [41]:
anslist[0]

(2)

In [147]:
b.column()*a.row()

[2 2 0]
[2 2 0]
[2 2 0]

In [92]:
b

(1, 1, 1)